In [1]:
import pickle
import os
Name = ['MeltedToast', 'Mithraaaa']
data = []
for name in Name:
    file_path = '/Users/yuxuan/Documents/AlphaGen/Replays/Match_Data/'+name
    replays = os.listdir(file_path)
    for item in replays:
        replay_path = file_path + '/' + item
        with open(replay_path,"rb") as file:
            data.append(pickle.load(file))

print(len(data))
# print(data[0])

10374


In [2]:
import numpy as np
print(data[0])

Data = np.array(data)
print(Data.shape)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, LSTM, Dense, TimeDistributed
from keras import backend as K
model = Sequential()
model.add(TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'), input_shape=(None, 25, 25, 2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dense(units=2, activation='linear'))
def loss_func(y_true, y_pred):
    return K.mean(K.sum(K.abs(y_pred-y_true),axis=-1))
model.compile(optimizer='adam', loss='mse')
model.summary()


NameError: name 't' is not defined

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)
model.save('model_full.h5')

In [ ]:
predicted_coords = model.predict(X_test)
